In [ ]:
%pip install dotenv
%pip install pandas
%pip install turfpy
%pip install rasterio fiona rasterstats
%pip install rasterio[s3]
%pip install pystac-client sat-search 
%pip install pypalettes
%pip install numpy
%pip install pyfonts

In [ ]:
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt

# City Municipal Districs
with open("./mo.geojson") as file:
    districts = gpd.read_file(file)

districts.set_crs(epsg=4326, inplace=True)
districts.explore()


In [ ]:
import shapely

total_bounds = gpd.GeoDataFrame(index=[0], crs='epsg:4326', geometry=[shapely.box(*districts.total_bounds)])
total_bounds.explore()

In [ ]:
from pystac_client import Client

# Browse USGS STAC
LandsatSTAC = Client.open("https://landsatlook.usgs.gov/stac-server", headers=[])

for collection in LandsatSTAC.get_collections():
    print(f"{collection.id}\t\t{collection.description}")

In [ ]:
import json
bbox = json.loads(total_bounds.to_json())['features'][0]['geometry']

# STAC Search
LandsatSearch = LandsatSTAC.search ( 
    intersects = bbox,
    datetime = '2024-06-28/2024-06-28',
    query =  ['eo:cloud_cover95'],
    collections = ["landsat-c2l2-sr"] )

Landsat_items = [i.to_dict() for i in LandsatSearch.get_items()]
print(f"{len(Landsat_items)} Landsat scenes fetched")
print(Landsat_items[0].keys())
for item in Landsat_items:
    print(item['id'])

In [ ]:
Landsat_item = Landsat_items[1]
print(Landsat_item['assets'].keys())
print(Landsat_item['assets']['red'].keys())
print(Landsat_item['assets']['red']["href"])
print(Landsat_item['assets']['red']["alternate"].keys())
print(Landsat_item['assets']['red']["alternate"]['s3'].keys())
print(Landsat_item['assets']['red']["alternate"]['s3']['href'])

In [ ]:
red_href = Landsat_item['assets']['red']['href']
nir_href = Landsat_item['assets']['nir08']['href']
red_s3 = Landsat_item['assets']['red']['alternate']['s3']['href']
nir_s3 = Landsat_item['assets']['nir08']['alternate']['s3']['href']
print(red_href)    
print(nir_href)
print(red_s3)    
print(nir_s3)

In [ ]:
# Download tiff using STAC link  (not working)

import requests
from dotenv import dotenv_values

config = dotenv_values(".env")
username = config['usgs_user']
token = config['usgs_token']

def download_file(session, url, filename):
    r = session.get(url, stream=True)
    if r.ok:
        with open(filename, 'wb') as file:
            # file.write(r.content)
            for chunk in r.iter_content(chunk_size=128):
                file.write(chunk)
        print(f"File {filename} downloaded successfully.")
    else:
        print(f"Failed to download file {url}. Status code: {r.status_code}")
    return filename

with requests.Session() as session:
    data = {
        'username': username, 
        'token': token
    }
    with session.post("https://m2m.cr.usgs.gov/api/api/json/stable/login-token", json=data) as response:
        api_token = response.json()['data']
        print(api_token)
        download_file(session, red_href, f"{Landsat_item['id']}_B4.html")
        download_file(session, nir_href, f"{Landsat_item['id']}_B5.TIF")


In [ ]:
# Direct Search

import requests
from dotenv import dotenv_values

config = dotenv_values(".env")
username = config['usgs_user']
token = config['usgs_token']

# USGS M2M API key
data = {
    'username': username, 
    'token': token
}
with requests.post("https://m2m.cr.usgs.gov/api/api/json/stable/login-token", json=data) as response:
    print(response.status_code)
    api_token = response.json()['data']

In [ ]:
import geojson

# Filters
total_bounds = districts.total_bounds

datasetName = "landsat-c2l2-sr"
spatialFilter =  {
    'filterType' : "mbr",
    'lowerLeft' : { 'latitude' : total_bounds[1], 'longitude' : total_bounds[0] },
    'upperRight' : { 'latitude' : total_bounds[3], 'longitude' : total_bounds[2] }
}
temporalFilter = {'start' : '2024-06-28', 'end' : '2024-06-28'}

In [ ]:
import requests
import json

# Datasets

headers = {'X-Auth-Token': api_token}   

payload = {
    'datasetName' : 'Landsat 8-9 OLI/TIRS C2 L2',
    'spatialFilter' : spatialFilter,
    'temporalFilter' : temporalFilter,
    "publicOnly": True
}

with (requests.post("https://m2m.cr.usgs.gov/api/api/json/stable/dataset-search", json.dumps(payload), headers = headers)) as response:
    print(response.status_code)
    dataset = json.loads(response.text)

print(json.dumps(dataset, indent=2))
# for dataset in datasets:
    # print(f"{dataset['datasetAlias']} {dataset['collectionName']}")


In [ ]:
data = {
    'datasetName' : dataset['data'][0]['datasetAlias'], 
    'maxResults' : 10,
    'startingNumber' : 0, 
    'sceneFilter' : {
        'spatialFilter' : spatialFilter,
        'acquisitionFilter' : temporalFilter,
    }
}

with (requests.post("https://m2m.cr.usgs.gov/api/api/json/stable/scene-search", json.dumps(data), headers = headers)) as response:
    print(response.status_code)
    scenes = json.loads(response.text)['data']['results']


for scene in scenes:
    print(f"{scene['displayId']} {scene['temporalCoverage']} {scene['spatialCoverage']}")

In [ ]:
spatialCoveragePolygons = []
for scene in scenes:
    # print(scene['spatialCoverage']['coordinates'][0])
    coordinates = scene['spatialCoverage']['coordinates'][0]

    spatialCoveragePolygons.append(shapely.geometry.Polygon(coordinates))

spatialCoverage = gpd.GeoSeries(spatialCoveragePolygons)
spatialCoverage.set_crs(epsg=4326, inplace=True)
spatialCoverage.explore()

In [ ]:
import requests

# Download Scene
scene = scenes[0]

data = {
    'datasetName' : dataset['data'][0]['datasetAlias'], 
    'entityIds' : scene['entityId'],
     "includeSecondaryFileGroups": True,
}

with (requests.post("https://m2m.cr.usgs.gov/api/api/json/stable/download-options", json.dumps(data), headers = headers)) as response:
    print(response.status_code)
    downloadOptions = json.loads(response.text)

# print(json.dumps(downloadOptions, indent=2))
for d in downloadOptions['data']:
    print(f"{d['id']} {d['displayId']} {d['productName']}, {d['productCode']}")

In [ ]:
product = next(p for p in downloadOptions['data'] if p['productCode'] == 'D694')
print(json.dumps(product, indent=2))

In [ ]:
for s in product['secondaryDownloads']:
    print(f"{s['downloadName']} {s['displayId']} {s['productCode']}, {s['downloadName']}")

In [ ]:
redBandProduct = next(x for x in product['secondaryDownloads'] if x['downloadName'] == 'SR_B4.TIF')
nirBandProduct = next(x for x in product['secondaryDownloads'] if x['downloadName'] == 'SR_B5.TIF')

products = [redBandProduct, nirBandProduct]
print(json.dumps(redBandProduct, indent=2))
print(json.dumps(nirBandProduct, indent=2))

In [ ]:
data = {
    "downloads": [
        {
            "entityId": redBandProduct['entityId'],
            "productId": redBandProduct['id'],
        },
        {
            "entityId": nirBandProduct['entityId'],
            "productId": nirBandProduct['id'],
        }
    ]
}

with (requests.post("https://m2m.cr.usgs.gov/api/api/json/stable/download-request", json.dumps(data), headers = headers)) as response:
    print(response.status_code)
    downloadRequest = json.loads(response.text)
    print(json.dumps(downloadRequest, indent=2))

In [ ]:
# Download tiff 
def download_file(url, filename):
    r = requests.get(url, stream=True)
    if r.ok:
        with open(filename, 'wb') as file:
            # file.write(r.content)
            for chunk in r.iter_content(chunk_size=128):
                file.write(chunk)
        file.close()
        print(f"File {filename} downloaded successfully.")
    else:
        print(f"Failed to download file {url}. Status code: {r.status_code}")
    return filename

# for ad in downloadRequest["data"]['availableDownloads']:
#     url = ad['url']
#     entityId = ad['entityId']
#     p = next(x for x in products if x['entityId'] == entityId)
#     filename = p['displayId']
#     download_file(url, filename)
ad = downloadRequest["data"]['availableDownloads'][1]
url = ad['url']
entityId = ad['entityId']
p = next(x for x in products if x['entityId'] == entityId)
filename = p['displayId']
download_file(url, filename)
    

In [ ]:
import rasterio

# View Details 
with rasterio.open(f"./{redBandProduct['displayId']}") as tiff:
    print(tiff)
    print(tiff.meta)
    print(tiff.profile)

with rasterio.open(f"./{nirBandProduct['displayId']}") as tiff:
    print(tiff)
    print(tiff.meta)
    print(tiff.profile)

In [ ]:
import matplotlib.pyplot as plt

# Calc NDVI
with rasterio.open(f"./{redBandProduct['displayId']}") as redBand, rasterio.open(f"./{nirBandProduct['displayId']}") as nirBand:
    red = redBand.read(1)
    nir = nirBand.read(1)

    # https://www.usgs.gov/faqs/how-do-i-use-a-scale-factor-landsat-level-2-science-products
    scale = 0.0000275
    offset = -0.2

    redValue = red * scale + offset
    nirValue = nir * scale + offset

    ndvi = (nirValue - redValue) / (nirValue + redValue)
    ndvi = ndvi.clip(-1, 1)

    redBand = tiff.profile
    redBand.update(dtype=rasterio.float32, count=1, compress="lzw")
    ndvi_filename = f"{product['displayId']}_SR_NDVI.TIF"
    # Save NDVI Raster
    with rasterio.open(ndvi_filename, "w", **redBand) as dst:
        dst.write(ndvi, 1)
        print(f"Raster data has been written to {ndvi_filename}")


#####################################

In [ ]:
import rasterio
import rasterio.plot
from pypalettes import add_cmap
from pyfonts import load_google_font

regular = load_google_font("Roboto")
bold = load_google_font("Roboto", weight="bold")

cmap = add_cmap(
    colors=[
        "#FFFFFF",
        "#CE7E45",
        "#DF923D",
        "#F1B555",
        "#FCD163",
        "#99B718",
        "#74A901",
        "#66A000",
        "#529400",
        "#3E8601",
        "#207401",
        "#056201",
        "#004C00",
        "#023B01",
        "#012E01",
        "#011D01",
        "#011301",
    ],
    cmap_type="continuous",
    name="NDVI",
)

districts.to_crs(epsg=32637, inplace=True)
with rasterio.open(ndvi_filename) as ndvi:
    fig, ax = plt.subplots(figsize=(8, 8))
    ax.axis("off")
    img = rasterio.plot.show(ndvi, ax=ax, cmap=cmap, vmin=0, vmax=1)
    im = img.get_images()[0]
    fig.colorbar(im, ax=ax, shrink=0.5)
    districts.dissolve().plot(ax=ax, edgecolor="white", facecolor="none", linewidth=1)
    fig.text(x=0.5, y=0.79, s="Vegetation index (NDVI) in Moscow", size=12, font=bold)
    fig.text(x=0.5, y=0.76, s="In June 2024", size=8, font=regular)
    fig.text(x=0.2, y=0.20, s="Map: Nikolai Groshkov\nSource: Landsat Collection 2 Level-2 image\ncourtesy of the U.S. Geological Survey.", 
        size=6, color="#909090", font=regular
    )


In [ ]:
from rasterstats import zonal_stats
import geopandas as gpd

# Calc and Join Zonal Stats
districts_stats = districts.join(
    gpd.GeoDataFrame(
        zonal_stats(
            vectors=districts['geometry'], 
            raster=ndvi_filename,
            stats=['mean']
        )
    ),
    how='left'
).rename(columns={"mean": "ndvi"})

districts_stats.to_file("mo.ndvi.stats.geojson", driver="GeoJSON")
districts_stats = districts_stats.sort_values(by='ndvi').reset_index(drop=True)
districts_stats

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

bins=[0, 0.2, 0.4, 0.6, 0.8, 1]
counts, bins = np.histogram(districts_stats["ndvi"], bins=bins)
colors = [cmap((val - min(bins)) / (max(bins) - min(bins))) for val in bins]

_, _, patches = plt.hist(bins[:-1], bins, weights=counts,edgecolor='white')
[patch.set_facecolor(colors[i]) for i, patch in enumerate(patches)]
[plt.gca().spines[pos].set_visible(False) for pos in ['right', 'top', 'left']]
plt.tick_params(axis='y', which='both', right=False, left=False, labelleft=False) 
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from pyfonts import load_google_font

regular = load_google_font("Roboto")
bold = load_google_font("Roboto", weight="bold")


fig, ax = plt.subplots(figsize=(8, 8))
ax.axis("off")

districts_stats.plot(ax=ax, column="ndvi", cmap=cmap, edgecolor="#e6e6e6", lw=0.3)

# Barplot
bins=[0, 0.2, 0.4, 0.6, 0.8, 1]
counts, bins = np.histogram(districts_stats["ndvi"], bins=bins)
colors = [cmap((val - min(bins)) / (max(bins) - min(bins))) for val in bins]

hist_ax = ax.inset_axes(bounds=[0.5, 0.1, 0.4, 0.15], zorder=-1)
_, _, patches = hist_ax.hist(bins[:-1], bins, weights=counts,edgecolor='white')
[patch.set_facecolor(colors[i]) for i, patch in enumerate(patches)]
hist_ax.spines[["top", "left", "right"]].set_visible(False)
hist_ax.set_xticks(bins)
hist_ax.set_yticks([])

fig.text(x=0.2, y=0.89, s="Vegetation index (NDVI) in Moscow", size=12, font=bold)
fig.text(x=0.2, y=0.86, s="By district, in June 2024", size=8, font=regular)
fig.text(x=0.2, y=0.13, s="Map: Nikolai Groshkov · Source: Landsat Collection 2 Level-2 image courtesy of the U.S. Geological Survey.", 
         size=6, color="#909090", font=regular
        )
plt.show()